In [56]:
import pandas as pd
import numpy as np
import spacy
import re

In [57]:
# df1 = pd.read_json("../Datasets/Official/train.json")
# df1

In [58]:
# df1[df1.isnull().any(axis = 1)]

In [59]:
df2 = pd.read_json("../Datasets/Official/test.json")
df2

,document,full_text,tokens,trailing_whitespace
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal..."
5,86,Cheese Startup - Learning Launch ​by Eladio Am...,"[Cheese, Startup, -, Learning, Launch, ​by, El...","[True, True, True, True, True, True, True, Fal..."
6,93,Silvia Villalobos\n\nChallenge:\n\nThere is a ...,"[Silvia, Villalobos, \n\n, Challenge, :, \n\n,...","[True, False, False, False, False, False, True..."
7,104,Storytelling The Path to Innovation\n\nDr Sak...,"[Storytelling, , The, Path, to, Innovation, \...","[True, False, True, True, True, False, False, ..."
8,112,Reflection – Learning Launch\n\nFrancisco Ferr...,"[Reflection, –, Learning, Launch, \n\n, Franci...","[True, True, True, False, False, True, False, ..."
9,123,Gandhi Institute of Technology and Management ...,"[Gandhi, Institute, of, Technology, and, Manag...","[True, True, True, True, True, True, False, Tr..."


In [60]:
df2[df2.isnull().any(axis = 1)]

,document,full_text,tokens,trailing_whitespace


In [66]:
df2_4_cleaned = [tok for tok in df2.iloc[4]['tokens'] if not re.search('.*\xa0.*',tok)]
df2.loc[4,'tokens'] = df2_4_cleaned

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
df3 = pd.read_csv("../Datasets/External/ai_data.csv",names=["text","label"],header = 0)
df3

In [62]:
nlp = spacy.load("en_core_web_sm")
text = df3['text'].loc[0]
doc = nlp(text)
for token in doc:
    print(token.text)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [63]:
df2_4_cleaned = [tok for tok in df2.iloc[4]['tokens'] if not re.search('.*\xa0.*',tok)]
with pd.option_context('mode.chained_assignment','warn'):
    df2.loc[4,'tokens'] = df2_4_cleaned

ValueError: Must have equal len keys and value when setting with an iterable